<a href="https://colab.research.google.com/github/Arkya187/Medical-Visual_Question_Answering/blob/main/VQA_using_ViLT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from PIL import Image
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset
from transformers import ViltProcessor, ViltForQuestionAnswering, TrainingArguments, Trainer
from transformers import default_data_collator
!pip install evaluate
import evaluate
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.4 MB/s eta 0:00:00


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mitanshuchakrawarty/medical-visual-question-answering")

print("Path to dataset files:", path)

100%|██████████| 16.1M/16.1M [00:01<00:00, 13.6MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/mitanshuchakrawarty/medical-visual-question-answering/versions/1


In [ ]:
metadata = pd.read_excel("/root/.cache/kagglehub/datasets/mitanshuchakrawarty/medical-visual-question-answering/versions/1/VQA_RAD Dataset Public.xlsx")
metadata

/usr/local/lib/python3.12/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,QID_unique,QID_para,QID_linked,IMAGEID_case,IMAGEID,IMAGEORGAN,EVALUATION,QUESTION,Q_REPHASE,Q_RELATION,Q_FRAMED,Q_TYPE,ANSWER,A_TYPE
0,0,freeform,03f451ca-de62-4617-9679-e836026a7642,https://medpix.nlm.nih.gov/case?id=48e1dd0e-85...,https://medpix.nlm.nih.gov/images/full/synpic5...,HEAD,not evaluated,Are regions of the brain infarcted?,NaN,NaN,NaN,PRES,Yes,CLOSED
1,1,freeform,06e26b2c-04b9-42bc-8e98-1de30a0f7682,https://medpix.nlm.nih.gov/case?id=b197277b-69...,https://medpix.nlm.nih.gov/images/full/synpic2...,CHEST,not evaluated,Are the lungs normal appearing?,NaN,NaN,NaN,ABN,No,CLOSED
2,2,freeform,0d0e8b6b-7753-4788-9b6d-dc7f25250c3f,https://medpix.nlm.nih.gov/case?id=b197277b-69...,https://medpix.nlm.nih.gov/images/full/synpic2...,CHEST,not evaluated,Is there evidence of a pneumothorax,NaN,NaN,NaN,PRES,No,CLOSED
3,3,freeform,0e90b6bc-265f-490b-a039-509b9907a3cb,https://medpix.nlm.nih.gov/case?id=19aa8a2b-35...,https://medpix.nlm.nih.gov/images/full/synpic2...,CHEST,given,What type of imaging does this not represent?,NaN,NaN,NaN,MODALITY,ultrasound,OPEN
4,4,freeform,1179f612-12e0-4dda-aee0-f14a5200be7b,https://medpix.nlm.nih.gov/case?id=b197277b-69...,https://medpix.nlm.nih.gov/images/full/synpic2...,CHEST,given,Is this a MRI of the chest?,NaN,NaN,NaN,MODALITY,no,CLOSED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2243,2243,para,6e01da36-ccda-4462-81c4-c7b5e495fb99,https://medpix.nlm.nih.gov/case?id=5dd90eed-4b...,https://medpix.nlm.nih.gov/images/full/synpic5...,HEAD,evaluated,"Is this image modality T1, T2, or FLAIR?","Is this a T1 weighted, T2 weighted, or FLAIR i...",strict agreement,What is/are the image weighting of the MRI?,MODALITY,FLAIR,CLOSED
2244,2244,freeform,9a4e1869-5c76-4af6-81e4-1864db18bbbe,https://medpix.nlm.nih.gov/case?id=5133d5d4-19...,https://medpix.nlm.nih.gov/images/full/synpic2...,CHEST,evaluated,Is this a PA film?,Is this film taken in a PA modality?,strict agreement,NaN,PLANE,Yes,CLOSED
2245,2245,para,9a4e1869-5c76-4af6-81e4-1864db18bbbe,https://medpix.nlm.nih.gov/case?id=5133d5d4-19...,https://medpix.nlm.nih.gov/images/full/synpic2...,CHEST,evaluated,Is this film taken in a PA modality?,Is this a PA film?,strict agreement,NaN,PLANE,Yes,CLOSED
2246,2246,freeform,e2c3565f-a137-406d-8ab0-de869c86cbee,https://medpix.nlm.nih.gov/case?id=187330b9-9d...,https://medpix.nlm.nih.gov/images/full/synpic3...,ABD,evaluated,What is the hypodensity located inside the ver...,What is the hypo-dense area seen in the verteb...,strict agreement,NaN,PRES,Nucleus Pulposus,OPEN


In [ ]:
sampledata = metadata[(metadata["ANSWER"] == "Yes") | (metadata["ANSWER"] == "No") | (metadata["ANSWER"] == "yes") | (metadata["ANSWER"] == "no") ]
sampledata.reset_index(inplace = True, drop = True)
sampledata

,QID_unique,QID_para,QID_linked,IMAGEID_case,IMAGEID,IMAGEORGAN,EVALUATION,QUESTION,Q_REPHASE,Q_RELATION,Q_FRAMED,Q_TYPE,ANSWER,A_TYPE
0,0,freeform,03f451ca-de62-4617-9679-e836026a7642,https://medpix.nlm.nih.gov/case?id=48e1dd0e-85...,https://medpix.nlm.nih.gov/images/full/synpic5...,HEAD,not evaluated,Are regions of the brain infarcted?,NaN,NaN,NaN,PRES,Yes,CLOSED
1,1,freeform,06e26b2c-04b9-42bc-8e98-1de30a0f7682,https://medpix.nlm.nih.gov/case?id=b197277b-69...,https://medpix.nlm.nih.gov/images/full/synpic2...,CHEST,not evaluated,Are the lungs normal appearing?,NaN,NaN,NaN,ABN,No,CLOSED
2,2,freeform,0d0e8b6b-7753-4788-9b6d-dc7f25250c3f,https://medpix.nlm.nih.gov/case?id=b197277b-69...,https://medpix.nlm.nih.gov/images/full/synpic2...,CHEST,not evaluated,Is there evidence of a pneumothorax,NaN,NaN,NaN,PRES,No,CLOSED
3,4,freeform,1179f612-12e0-4dda-aee0-f14a5200be7b,https://medpix.nlm.nih.gov/case?id=b197277b-69...,https://medpix.nlm.nih.gov/images/full/synpic2...,CHEST,given,Is this a MRI of the chest?,NaN,NaN,NaN,MODALITY,no,CLOSED
4,6,freeform,27169439-b749-4b2f-8abe-bfb57a46ce53,https://medpix.nlm.nih.gov/case?id=19aa8a2b-35...,https://medpix.nlm.nih.gov/images/full/synpic2...,CHEST,not evaluated,Is the trachea midline?,NaN,NaN,NaN,PRES,yes,CLOSED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187,2236,freeform,2f181109-0890-4447-b2fd-645176ba9aa8,https://medpix.nlm.nih.gov/case?id=693c5942-66...,https://medpix.nlm.nih.gov/images/full/synpic5...,HEAD,evaluated,Is there a midline shift?,Is there structural deviation from the midline?,strict agreement,Is there midline shift?,PRES,No,CLOSED
1188,2237,para,2f181109-0890-4447-b2fd-645176ba9aa8,https://medpix.nlm.nih.gov/case?id=693c5942-66...,https://medpix.nlm.nih.gov/images/full/synpic5...,HEAD,evaluated,Is there structural deviation from the midline?,Is there a midline shift?,strict agreement,Is there midline shift?,PRES,No,CLOSED
1189,2240,freeform,436c2fd7-b598-437a-9d84-ff2778cc8250,https://medpix.nlm.nih.gov/case?id=1aadbdc9-95...,https://medpix.nlm.nih.gov/images/full/synpic3...,CHEST,not evaluated,Is there decreased aeration at the apex?,Is the apical aeration normal or decreased?,not similar,NaN,PRES,Yes,CLOSED
1190,2244,freeform,9a4e1869-5c76-4af6-81e4-1864db18bbbe,https://medpix.nlm.nih.gov/case?id=5133d5d4-19...,https://medpix.nlm.nih.gov/images/full/synpic2...,CHEST,evaluated,Is this a PA film?,Is this film taken in a PA modality?,strict agreement,NaN,PLANE,Yes,CLOSED


In [ ]:
labels = np.zeros((len(sampledata),))
for i in range(len(sampledata)):
    if (sampledata["ANSWER"][i] == "Yes") | (sampledata["ANSWER"][i] == "yes"):
        labels[i] = 1
    elif (sampledata["ANSWER"][i] == "No") | (sampledata["ANSWER"][i] == "no"):
        labels[i] = 0
    else:
        labels[i] = None

In [ ]:
IMGDIR = "/root/.cache/kagglehub/datasets/mitanshuchakrawarty/medical-visual-question-answering/versions/1/VQA_RAD Image Folder/"

addresses = []
for i in range(len(sampledata)):
    addresses.append(IMGDIR + sampledata["IMAGEID"][i][39:])
addresses[:10]

['/root/.cache/kagglehub/datasets/mitanshuchakrawarty/medical-visual-question-answering/versions/1/VQA_RAD Image Folder/synpic54610.jpg',
 '/root/.cache/kagglehub/datasets/mitanshuchakrawarty/medical-visual-question-answering/versions/1/VQA_RAD Image Folder/synpic29265.jpg',
 '/root/.cache/kagglehub/datasets/mitanshuchakrawarty/medical-visual-question-answering/versions/1/VQA_RAD Image Folder/synpic29265.jpg',
 '/root/.cache/kagglehub/datasets/mitanshuchakrawarty/medical-visual-question-answering/versions/1/VQA_RAD Image Folder/synpic29265.jpg',
 '/root/.cache/kagglehub/datasets/mitanshuchakrawarty/medical-visual-question-answering/versions/1/VQA_RAD Image Folder/synpic28602.jpg',
 '/root/.cache/kagglehub/datasets/mitanshuchakrawarty/medical-visual-question-answering/versions/1/VQA_RAD Image Folder/synpic28602.jpg',
 '/root/.cache/kagglehub/datasets/mitanshuchakrawarty/medical-visual-question-answering/versions/1/VQA_RAD Image Folder/synpic28602.jpg',
 '/root/.cache/kagglehub/datasets/

In [ ]:
image_paths = addresses
questions = sampledata['QUESTION'].astype(str).tolist()

In [ ]:
from transformers import ViltModel
import torch.nn as nn

MODEL_NAME = "dandelin/vilt-b32-mlm"

processor = ViltProcessor.from_pretrained(MODEL_NAME)

class ViltForBinaryClassification(nn.Module):
    def __init__(self, model_name, num_labels=2):
        super().__init__()
        self.vilt = ViltModel.from_pretrained(model_name)
        self.classifier = nn.Linear(self.vilt.config.hidden_size, num_labels)
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, input_ids=None, pixel_values=None, attention_mask=None, labels=None):
        outputs = self.vilt(
            input_ids=input_ids,
            pixel_values=pixel_values,
            attention_mask=attention_mask,
        )
        pooled_output = outputs.pooler_output
        logits = self.classifier(pooled_output)
        loss = None
        if labels is not None:
            loss = self.loss_fn(logits, labels)
        return {"loss": loss, "logits": logits}

model = ViltForBinaryClassification(MODEL_NAME)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

In [ ]:
class VQADataset(Dataset):
    def __init__(self, image_paths, questions, labels, processor):
        self.image_paths = image_paths
        self.questions = questions
        self.labels = labels
        self.processor = processor

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB").resize((384, 384))
        question = self.questions[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.long)

        encoding = self.processor(
            text=question,
            images=image,
            return_tensors="pt",
            truncation=True,
            padding="max_length",
            max_length=40,
        )

        encoding = {k: v.squeeze(0) for k, v in encoding.items()}
        encoding["labels"] = label
        return encoding

In [ ]:
train_paths, val_paths, train_qs, val_qs, train_lbls, val_lbls = train_test_split(
    image_paths, questions, labels, test_size=0.2, random_state=42, stratify=labels
)

train_dataset = VQADataset(train_paths, train_qs, train_lbls, processor)
val_dataset = VQADataset(val_paths, val_qs, val_lbls, processor)

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, zero_division=0)
    prec = precision_score(labels, preds, zero_division=0)
    rec = recall_score(labels, preds, zero_division=0)
    return {"accuracy": acc, "f1": f1, "precision": prec, "recall": rec}

In [ ]:
training_args = TrainingArguments(
    output_dir="./vilt_vqa_rad",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=20,
    learning_rate=5e-5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    fp16=True
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=default_data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: arkyachatterjee8 (arkyachatterjee8-nit-raipur) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.684200,0.946122,0.518828,0.033613,1.000000,0.017094
2,0.688500,0.820384,0.665272,0.619048,0.698925,0.555556
3,0.731700,1.413780,0.698745,0.666667,0.727273,0.615385
4,0.423700,1.537275,0.702929,0.666667,0.739583,0.606838
5,0.317200,1.544736,0.711297,0.698690,0.714286,0.683761
6,0.114600,2.046922,0.665272,0.595960,0.728395,0.504274
7,0.160300,1.792313,0.694561,0.654028,0.734043,0.589744
8,0.128000,2.319676,0.677824,0.688259,0.653846,0.726496
9,0.050600,2.173641,0.694561,0.694561,0.680328,0.709402
10,0.101800,2.319660,0.694561,0.702041,0.671875,0.735043


TrainOutput(global_step=4770, training_loss=0.3586764589045783, metrics={'train_runtime': 1336.2581, 'train_samples_per_second': 7.132, 'train_steps_per_second': 3.57, 'total_flos': 0.0, 'train_loss': 0.3586764589045783, 'epoch': 10.0})

In [ ]:
metrics = trainer.evaluate()
print(metrics)

{'eval_loss': 2.3197238445281982, 'eval_accuracy': 0.694560669456067, 'eval_f1': 0.7020408163265306, 'eval_precision': 0.671875, 'eval_recall': 0.7350427350427351, 'eval_runtime': 7.7009, 'eval_samples_per_second': 31.035, 'eval_steps_per_second': 15.583, 'epoch': 10.0}


In [ ]:
trainer.save_model("./vilt_vqa_rad_best")
processor.save_pretrained("./vilt_vqa_rad_best")

[]

In [ ]:
from transformers import ViltProcessor, ViltForQuestionAnswering
processor = ViltProcessor.from_pretrained("./vilt_vqa_rad_best")
model = ViltForQuestionAnswering.from_pretrained("./vilt_vqa_rad_best")

Some weights of ViltForQuestionAnswering were not initialized from the model checkpoint at ./vilt_vqa_rad_best and are newly initialized: ['classifier.0.bias', 'classifier.0.weight', 'classifier.1.bias', 'classifier.1.weight', 'classifier.3.bias', 'classifier.3.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
num_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {num_params:,}")
print(f"Approx model size: {num_params * 4 / (1024**2):.2f} MB (in FP32)")

Total parameters: 112,782,338
Approx model size: 430.23 MB (in FP32)


In [ ]:
import time
import torch
from PIL import Image

image = Image.open(addresses[0]).convert("RGB").resize((384,384))
question = sampledata["QUESTION"][0]

# Explicit truncation + GPU move
inputs = processor(image, question, return_tensors="pt", truncation=True, max_length=40)
inputs = {k: v.to("cuda") for k, v in inputs.items()}

model.to("cuda")
model.eval()

# Warmup
_ = model(**inputs)

# Measure inference time
start = time.time()
with torch.no_grad():
    outputs = model(**inputs)
torch.cuda.synchronize()
end = time.time()

print(f"Inference time per sample: {(end - start)*1000:.2f} ms")

Inference time per sample: 25.94 ms
